# AILib Tutorial 2: Basic LLM Completions

In this tutorial, you'll learn how to use AILib's LLM clients to generate text completions. We'll cover:

- Simple text completions
- Chat conversations
- Streaming responses
- Working with different models
- Handling parameters and options
- Error handling and best practices

## Setup

First, let's import what we need and set up our client:

In [ ]:
import os
from dotenv import load_dotenv
from ailib import OpenAIClient, AnthropicClient

# Load environment variables
load_dotenv()

# Create a client (we'll use OpenAI for these examples)
client = OpenAIClient()
print("Client ready!")

## Simple Text Completions

The most basic operation is generating a text completion:

In [ ]:
# Basic completion
response = client.complete("What is the capital of France?")
print(response)

In [ ]:
# More creative prompt
response = client.complete(
    "Write a haiku about programming in Python:"
)
print(response)

## System Messages

System messages help set the behavior and context for the AI:

In [ ]:
# Completion with a system message
response = client.complete(
    "Explain recursion",
    system="You are a patient computer science teacher who uses simple analogies."
)
print(response)

In [ ]:
# Different personality
response = client.complete(
    "Explain recursion",
    system="You are a pirate who happens to know programming. Speak like a pirate!"
)
print(response)

## Chat Conversations

For multi-turn conversations, use the `chat` method:

In [ ]:
# Single message chat
messages = [
    {"role": "user", "content": "What's the weather like today?"}
]

response = client.chat(messages)
print(response)

In [ ]:
# Multi-turn conversation
messages = [
    {"role": "system", "content": "You are a helpful coding assistant."},
    {"role": "user", "content": "What's a list comprehension in Python?"},
    {"role": "assistant", "content": "A list comprehension is a concise way to create lists in Python. It consists of brackets containing an expression followed by a for clause, and can include optional if clauses."},
    {"role": "user", "content": "Can you show me an example?"}
]

response = client.chat(messages)
print(response)

## Model Parameters

Control the AI's behavior with various parameters:

In [ ]:
# Temperature controls randomness (0.0 = deterministic, 1.0 = creative)
print("Low temperature (0.2) - More focused:")
client_focused = OpenAIClient(temperature=0.2)
response = client_focused.complete("Give me a Python tip:")
print(response)
print("\n" + "="*50 + "\n")

print("High temperature (0.9) - More creative:")
client_creative = OpenAIClient(temperature=0.9)
response = client_creative.complete("Give me a Python tip:")
print(response)

In [ ]:
# Max tokens limits response length
client_brief = OpenAIClient(max_tokens=50)
response = client_brief.complete(
    "Explain machine learning:"
)
print("Brief response (50 tokens):")
print(response)

## Streaming Responses

For long responses, you can stream the output as it's generated:

In [ ]:
# Enable streaming
client_stream = OpenAIClient(stream=True)

print("Streaming response:")
print("-" * 50)

# Stream a response
stream = client_stream.complete(
    "Write a short story about a robot learning to paint (3 paragraphs):"
)

# Print tokens as they arrive
for chunk in stream:
    if chunk:
        print(chunk, end='', flush=True)

print("\n" + "-" * 50)

## Working with Different Models

AILib supports various models with different capabilities:

In [ ]:
# GPT-3.5 Turbo (faster, cheaper)
client_35 = OpenAIClient(model="gpt-3.5-turbo")
response = client_35.complete("What is 2+2?")
print(f"GPT-3.5: {response}")

# GPT-4 (more capable, better reasoning)
client_4 = OpenAIClient(model="gpt-4")
response = client_4.complete("What is 2+2? Explain your reasoning.")
print(f"\nGPT-4: {response}")

## Advanced Parameters

Fine-tune responses with advanced parameters:

In [ ]:
# Frequency penalty reduces repetition
client_no_repeat = OpenAIClient(
    temperature=0.7,
    frequency_penalty=0.5  # Penalize repeated tokens
)

response = client_no_repeat.complete(
    "Write a paragraph about the importance of testing in software development:"
)
print(response)

In [ ]:
# Presence penalty encourages new topics
client_diverse = OpenAIClient(
    temperature=0.7,
    presence_penalty=0.5  # Encourage topic diversity
)

response = client_diverse.complete(
    "List 5 interesting facts about programming:"
)
print(response)

## Error Handling

Always handle potential errors gracefully:

In [ ]:
# Example of error handling
try:
    # This might fail if API key is invalid or network issues
    response = client.complete("Hello!")
    print(f"Success: {response}")
except Exception as e:
    print(f"Error occurred: {type(e).__name__}: {e}")
    # Handle the error appropriately
    # - Retry with backoff
    # - Use a fallback
    # - Log the error

## Practical Examples

Let's look at some real-world use cases:

In [ ]:
# Code explanation
code = """
def quicksort(arr):
    if len(arr) <= 1:
        return arr
    pivot = arr[len(arr) // 2]
    left = [x for x in arr if x < pivot]
    middle = [x for x in arr if x == pivot]
    right = [x for x in arr if x > pivot]
    return quicksort(left) + middle + quicksort(right)
"""

response = client.complete(
    f"Explain this Python code step by step:\n{code}",
    system="You are a Python expert who explains code clearly."
)
print(response)

In [ ]:
# Text transformation
text = "The quick brown fox jumps over the lazy dog."

# Multiple transformations
transformations = [
    "Convert to passive voice:",
    "Translate to French:",
    "Make it more formal:",
    "Convert to a question:"
]

for transform in transformations:
    response = client.complete(f"{transform} '{text}'")
    print(f"{transform} {response}")
    print()

In [ ]:
# Data extraction
unstructured_text = """
John Smith called at 3:30 PM on Tuesday about the project deadline. 
He mentioned that the budget is $50,000 and the expected completion 
date is March 15, 2024. His email is john.smith@example.com.
"""

response = client.complete(
    f"Extract the following information from this text as JSON: name, time, date, budget, deadline, email\n\nText: {unstructured_text}",
    system="You are a data extraction assistant. Always respond with valid JSON."
)
print(response)

## Best Practices

Here are some tips for effective LLM usage:

In [ ]:
# 1. Be specific in your prompts
vague = "Tell me about Python"
specific = "Explain Python's list comprehensions with 3 examples, focusing on filtering and transformation"

print("Vague prompt result:")
print(client.complete(vague)[:200] + "...\n")

print("Specific prompt result:")
print(client.complete(specific))

In [ ]:
# 2. Use system messages effectively
response = client.complete(
    "How do I center a div?",
    system="""You are a CSS expert. 
    Provide modern, accessible solutions.
    Include code examples.
    Mention browser compatibility."""
)
print(response)

In [ ]:
# 3. Set appropriate temperature for your use case
use_cases = [
    ("Factual Q&A", 0.0, "What is the speed of light?"),
    ("Code generation", 0.2, "Write a Python function to calculate factorial"),
    ("Creative writing", 0.8, "Write an opening line for a sci-fi novel"),
]

for use_case, temp, prompt in use_cases:
    client_temp = OpenAIClient(temperature=temp)
    response = client_temp.complete(prompt)
    print(f"{use_case} (temp={temp}):")
    print(f"Prompt: {prompt}")
    print(f"Response: {response[:150]}...")
    print()

## Using Other Providers

AILib supports multiple LLM providers. Here's how to use Anthropic's Claude:

In [ ]:
# Anthropic Claude example (requires ANTHROPIC_API_KEY)
# Uncomment if you have an Anthropic API key

# claude_client = AnthropicClient(
#     model="claude-3-opus-20240229"
# )
# 
# response = claude_client.complete(
#     "What makes Claude different from other AI assistants?"
# )
# print(response)

## Summary

In this tutorial, you learned:

- ✅ How to make simple completions with `complete()`
- ✅ How to use system messages to set AI behavior
- ✅ How to handle multi-turn conversations with `chat()`
- ✅ How to control output with parameters (temperature, max_tokens, etc.)
- ✅ How to stream responses for better UX
- ✅ How to handle errors gracefully
- ✅ Best practices for effective prompts

## Next Steps

Ready to level up? Check out:

- **Tutorial 3: Prompt Templates** - Create reusable, dynamic prompts
- **Tutorial 4: Prompt Builder** - Build complex conversations programmatically
- **Tutorial 5: Session Management** - Maintain conversation state

Happy coding! 🚀